In [12]:
from easyquant import MongoIo
from easyquant.indicator.base import *
import easyquotation
import math
import numpy as np
import pandas as pd
from easyquant import MongoIo
# from easyquant.qafetch import QATdx as tdx
from easyquant.indicator.base import *
import pandas as pd
from easyquant import EasyTime
from datetime import datetime, time
from ctypes import * 
# import datetime
import math
import numpy as np
import pandas as pd
import talib
from ctypes import * 
import QUANTAXIS as QA

# import time

In [13]:
m=MongoIo()
source=easyquotation.use('sina')

In [14]:
table = 'st_orders-%s' % 'tdx_yhzc'
code='002614'

In [21]:
# today
code='300063'
data=m.get_stock_day(code)


In [22]:
newdata=source.stocks(code)[code]

In [23]:
now_price = newdata['now']

In [24]:

def new_df(df_day, data, now_price):
    code = data['code']
    # now_vol = data['volume']
    last_time = pd.to_datetime(data['datetime'][0:10])
    # print("code=%s, data=%s" % (self.code, self._data['datetime']))
    # df_day = data_buf_day[code]
    # df_day.loc[last_time]=[0 for x in range(len(df_day.columns))]
    df_day.at[(last_time, code), 'open'] = data['open']
    df_day.at[(last_time, code), 'high'] = data['high']
    df_day.at[(last_time, code), 'low'] = data['low']
    df_day.at[(last_time, code), 'close'] = now_price
    df_day.at[(last_time, code), 'volume'] = data['volume']
    df_day.at[(last_time, code), 'amount'] = data['amount']
    return df_day


In [160]:
def tdx_dqe_cfc_A1(data, sort=False):
    # 选择／排序
    C = data.close
    O = data.open
    JC =IF(ISLASTBAR(C), O, C)
    MC = (0.3609454219 * JC - 0.03309329629 * REF(C, 1) - 0.04241822779 * REF(C, 2) - 0.026737249 * REF(C, 3) \
           - 0.007010041271 * REF(C, 4) - 0.002652859952 * REF(C, 5) - 0.0008415042966 * REF(C, 6) \
           - 0.0002891931964 * REF(C, 7) - 0.0000956265934 * REF(C, 8) - 0.0000321286052 * REF(C, 9) \
           - 0.0000106773454 * REF(C, 10) - 0.0000035457562 * REF(C, 11) -- 0.0000011670713 * REF(C, 12)) / (1 - 0.7522406533)
    # 竞价涨幅 := (DYNAINFO(4) / DYNAINFO(3) - 1) * 100;
    竞价涨幅 = (C / REF(C, 1) - 1) * 100
    # ST := STRFIND(stkname, 'ST', 1) > 0;
    # S := STRFIND(stkname, 'S', 1) > 0;
    # 停牌 := (DYNAINFO(4)=0);
    #
    # 附加条件 := (not (ST) and not (S) and NOT(停牌)) * (竞价涨幅 < 9.85) * (竞价涨幅 > (0));
    附加条件 = IFAND(竞价涨幅 < 9.85, 竞价涨幅 > 0, 1, 0)
#     if sort:
    刀 = (MC - JC) / JC * 1000
#     else:
#         刀 = (MC - JC) / JC * 1000 * 附加条件

    return 刀

In [161]:
def tdx_dqe_cfc_A12(data, sort=False):
    # 选择／排序
    C = data.close
    O = data.open
    JC = IF(ISLASTBAR(C), O, C)
    MC = (0.3609454219 * JC - 0.03309329629 * REF(C, 1) - 0.04241822779 * REF(C, 2) - 0.026737249 * REF(C, 3) \
           - 0.007010041271 * REF(C, 4) - 0.002652859952 * REF(C, 5) - 0.0008415042966 * REF(C, 6) \
           - 0.0002891931964 * REF(C, 7) - 0.0000956265934 * REF(C, 8) - 0.0000321286052 * REF(C, 9) \
           - 0.0000106773454 * REF(C, 10) - 0.0000035457562 * REF(C, 11) -- 0.0000011670713 * REF(C, 12)) / (1 - 0.7522406533)
    # 竞价涨幅 := (DYNAINFO(4) / DYNAINFO(3) - 1) * 100;
    竞价涨幅 = (O / REF(C, 1) - 1) * 100
    # ST := STRFIND(stkname, 'ST', 1) > 0;
    # S := STRFIND(stkname, 'S', 1) > 0;
    # 停牌 := (DYNAINFO(4)=0);
    #
    # 附加条件 := (not (ST) and not (S) and NOT(停牌)) * (竞价涨幅 < 9.85) * (竞价涨幅 > (0));
    附加条件 = IFAND(竞价涨幅 < 9.85, 竞价涨幅 > 0, 1, 0)
#     if sort:
    刀 = (MC - JC) / JC * 1000
#     else:
#         刀 = (MC - JC) / JC * 1000 * 附加条件

    return 刀

In [162]:
data = new_df(data.copy(), newdata, now_price)

In [207]:
# today
code='002759'
data=m.get_stock_day(code)
newdata=source.stocks(code)[code]
now_price = newdata['now']
data = new_df(data.copy(), newdata, now_price)
data.iat[-1,3]=newdata['open']
data.tail()
C = data.close
O = data.open
V = data.volume
H = data.high
L = data.low


In [208]:
TJ1=IFAND(O > REF(MA(C, 5), 1),  O > REF(MA(C, 10), 1), True, False).tail()
TJ2=(COUNT(REF(V, 1) / REF(V, 2) > 6, 10) == 0).tail()
ccc=tdx_dqe_cfc_A1(data).tail()
data['A1']=ccc
data['tj1']=IFAND(O > REF(MA(C, 5), 1),  O > REF(MA(C, 10), 1), True, False)
data['tj2']=(COUNT(REF(V, 1) / REF(V, 2) > 6, 10) == 0)
# result


In [209]:
data.tail()

,,open,high,low,close,volume,amount,A1,tj1,tj2
date,code,,,,,,,,,
2020-11-02,002759,16.05,17.80,15.98,17.42,2.240450e+05,375140832.0,12.382622,False,True
2020-11-03,002759,17.16,17.28,15.68,15.78,2.518350e+05,411066112.0,-34.031954,True,True
2020-11-04,002759,15.70,16.50,15.31,15.99,1.614650e+05,257307296.0,-19.056647,False,True
2020-11-05,002759,16.00,17.59,15.72,17.59,2.863450e+05,486238368.0,33.194479,False,True
2020-11-06,002759,18.14,18.50,16.61,18.14,4.499267e+08,25643556.0,40.854449,True,True


In [206]:
data.tail()

,,open,high,low,close,volume,amount,A1,tj1,tj2
date,code,,,,,,,,,
2020-11-02,300538,20.96,21.09,20.21,20.26,2.187600e+04,44767040.0,-38.249502,False,True
2020-11-03,300538,20.51,20.93,20.29,20.88,1.637000e+04,34001340.0,-7.105863,False,True
2020-11-04,300538,20.84,21.16,20.44,20.70,1.700400e+04,35326988.0,-2.517811,False,True
2020-11-05,300538,20.91,21.50,20.85,21.42,2.203200e+04,46691732.0,14.573609,False,True
2020-11-06,300538,22.60,25.70,21.70,22.60,2.588316e+08,10506785.0,33.348295,True,True


In [203]:
data.tail()

,,open,high,low,close,volume,amount,A1,tj1,tj2
date,code,,,,,,,,,
2020-11-02,300612,28.61,28.92,28.00,28.85,6.313200e+04,180467072.0,-25.341236,False,True
2020-11-03,300612,29.06,29.90,28.52,29.04,6.166800e+04,179565408.0,-4.895795,False,True
2020-11-04,300612,29.04,29.09,27.47,27.68,7.382400e+04,207252672.0,-21.126818,False,True
2020-11-05,300612,28.51,33.10,28.51,30.45,1.659820e+05,510805696.0,27.537533,False,True
2020-11-06,300612,31.22,32.70,30.50,31.22,5.102551e+08,16217014.0,33.468828,True,True


In [105]:
300208    14.250414
300538    33.348295
300050    20.557449
002614    24.505901
300063    30.333110

SyntaxError: invalid syntax (<ipython-input-105-6da975865391>, line 1)

In [154]:
# back:
code='300430'
data=m.get_stock_day(code, st_end='2020-11-05')

C = data.close
O = data.open
V = data.volume
H = data.high
L = data.low
# data.tail()

In [155]:
data=data.copy()
ccc=tdx_dqe_cfc_A12(data).tail()
data['A1']=ccc
data['tj1']=IFAND(O > REF(MA(C, 5), 1),  O > REF(MA(C, 10), 1), True, False)
data['tj2']=(COUNT(REF(V, 1) / REF(V, 2) > 6, 10) == 0)
# result
data.tail()

,,open,high,low,close,volume,amount,A1,tj1,tj2
date,code,,,,,,,,,
2020-10-30,300430,11.05,11.83,10.80,11.28,205857.0,230976608.0,30.732060,True,True
2020-11-02,300430,11.01,12.45,10.62,11.85,193750.0,225403616.0,36.805667,True,True
2020-11-03,300430,12.22,13.85,11.26,12.32,317865.0,386291296.0,37.046603,True,True
2020-11-04,300430,11.99,12.18,11.12,11.59,229040.0,267970624.0,-6.498246,True,True
2020-11-05,300430,13.91,13.91,11.81,12.25,263799.0,328365632.0,66.650803,True,True


In [223]:
code='300612'
dataInfo = m.get_stock_info(code)
dataInfo.jinglirun[0]

-112066005.859375

In [211]:
300364	7.62	8.49	7.38	7.92	787380.0	621797440.0	48.325356
300464	20.50	22.88	20.08	22.30	249674.0	533748384.0	0.000000	True	True
300689	48.51	53.56	47.11	49.70	51398.0	258483824.0	25.845675

SyntaxError: invalid syntax (<ipython-input-211-d05762d70d5f>, line 1)

In [ ]:
dataInfo = mongo.get_stock_info(code)